## What’s your favorite movie or TV show? - Tutorial on Webscraping IMDB

In this blog post, we are going to look at an interesting application of webscraping!!! We would like to develop a recommender system to see which movie or TV shows share actors with your favorite movie or show. Here is a link to my project repository: 

In [ ]:
import scrapy

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("results.csv")

In [3]:
df = df[["movie_or_TV_name"]]
df = df.rename(columns={"movie_or_TV_name":"Movie or TV Show"})

In [4]:
recommend = df.value_counts()
recommend = recommend.reset_index(name = "Number of Shared Actors")

In [6]:
recommend = recommend.head(10)
recommend

,Movie or TV Show,Number of Shared Actors
0,Little Women,131
1,Old Hearts Cafe,21
2,Castle Rock,20
3,The Oscars,16
4,Kromz Homes: Dreams,15
5,Entertainment Tonight,15
6,Celebrity Page,15
7,Hollywood Insider,14
8,Don't Look Up,14
9,Spenser Confidential,11


pandas.core.frame.DataFrame